In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tempfile
import zipfile

temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory 'C:\\Users\\andre\\AppData\\Local\\Temp\\tmpywy8t_7d'>


In [3]:
with zipfile.ZipFile('dataset.zip', 'r') as zip:
    zip.extractall(temp_dir.name)

In [4]:
#estrutura da rede neural
clf = Sequential()
clf.add(InputLayer(shape=(64, 64, 3)))#imagens 64x64 e coloridas(3)
clf.add(Conv2D(filters= 32, kernel_size=(3,3), activation='relu'))#camadas de convolução(32), kernel_size(tamanho do detector de caracteristicas)
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size=(2,2)))

clf.add(Conv2D(filters= 32, kernel_size=(3,3), activation='relu'))#camadas de convolução(32), kernel_size(tamanho do detector de caracteristicas)
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size=(2,2)))

clf.add(Flatten())

clf.add(Dense(units=128, activation='relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units=128, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(units=1, activation='sigmoid'))#1 pois é classificação binaria

In [5]:
clf.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 829,985 (3.17 MB)

 Trainable params: 829,857 (3.17 MB)

 Non-trainable params: 128 (512.00 B)

In [6]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
gerador_treinamento = ImageDataGenerator(rescale=1./255, rotation_range=7,
                                         horizontal_flip=True, shear_range=0.2,
                                         height_shift_range=0.07, zoom_range=0.2)

In [8]:
gerador_teste = ImageDataGenerator(rescale=1./255)

In [9]:
base_treinamento = gerador_treinamento.flow_from_directory(f'{temp_dir.name}/dataset/training_set',
                                                           target_size=(64,64),
                                                           batch_size=32, class_mode='binary')

Found 4000 images belonging to 2 classes.


In [10]:
base_teste = gerador_teste.flow_from_directory(f'{temp_dir.name}/dataset/test_set',
                                                target_size=(64,64),
                                                batch_size=32, class_mode='binary')

Found 1000 images belonging to 2 classes.


In [11]:
clf.fit(base_treinamento, epochs=50, validation_data=base_teste)

Epoch 1/50


c:\Users\andre\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 47s 340ms/step - accuracy: 0.5461 - loss: 0.9185 - val_accuracy: 0.5230 - val_loss: 0.7817
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 155ms/step - accuracy: 0.6383 - loss: 0.6699 - val_accuracy: 0.5450 - val_loss: 0.7571
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 141ms/step - accuracy: 0.6229 - loss: 0.6523 - val_accuracy: 0.5800 - val_loss: 0.7143
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 162ms/step - accuracy: 0.6668 - loss: 0.6101 - val_accuracy: 0.5830 - val_loss: 0.7171
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 161ms/step - accuracy: 0.6610 - loss: 0.6123 - val_accuracy: 0.6870 - val_loss: 0.5858
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 157ms/step - accuracy: 0.7081 - loss: 0.5800 - val_accuracy: 0.6480 - val_loss: 0.6179
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 151ms/step - accuracy: 0.7212 - loss: 0.5551 - val_accuracy: 0.6870 - val_loss: 0.5930
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 143ms/step - accuracy: 0.7245 - loss: 0.5504 - val